In [1]:
import pickle, lzma
%load_ext Cython

In [3]:

latticeData = pickle.load(lzma.open("yeast-lattice.2.pkl.xz", "rb"))
print('latticeData:', latticeData.keys())
siteMap = {n:i for i,n in enumerate(latticeData['names'])}
def boolLattice(x):
    return latticeData['lattice'] == siteMap[x]


extracellular = boolLattice("extracellular") | boolLattice("cellWall")
obstructions = boolLattice("nuclearEnvelope") | boolLattice("mitochondria") | boolLattice('vacuole')

ribosomes = boolLattice("ribosomes")
membrane = boolLattice("plasmaMembrane")
nucleus = boolLattice("nucleoplasm") | boolLattice("nuclearPores")
cytoplasm = boolLattice("cytoplasm")

# decimation = latticeData['decimation']

latticeData: dict_keys(['latticeSpacing', 'decimation', 'names', 'lattice'])


get relative cubes of different components

In [4]:
# original cytoplasm
import numpy as np
print(f"number of ribosomes: {np.sum(ribosomes)}")

ori_cytoplasm = cytoplasm | ribosomes
print(f"original cytoplasm: {np.sum(ori_cytoplasm)}")
# caculate the size of entire cell
print(f"entire cell: {np.sum(boolLattice('extracellular')!= 1)}")

number of ribosomes: 180000
original cytoplasm: 1086898
entire cell: 1701571


## Now redistribute the ribosomes to the cytoplasm region

In [5]:
%%cython
# cython: boundscheck=False
# cython: cdivision=True
# cython: wraparound=False

from libc.stdlib cimport *
from posix.stdlib cimport *
from libc.stdint cimport *

cdef int64_t randIntLessThan(int64_t n):
    cdef int64_t x = random()
    while x >= RAND_MAX - (RAND_MAX % n):
          x = random()
    return x % n

def placeRibosomes(int n, int64_t[:,:,:] avail, int64_t[:,:,:] rib):
    cdef int64_t nx, ny, nz, i, j, k
    nx,ny,nz = avail.shape[:3]
    
    while n > 0:
        i = 1+randIntLessThan(nx-2)
        j = 1+randIntLessThan(ny-2)
        k = 1+randIntLessThan(nz-2)
        if avail[i,j,k]:
            # make sure ribosomes are not connect to each other
            avail[i-1:i+2,j,k] = 0
            avail[i,j-1:j+2,k] = 0
            avail[i,j,k-1:k+2] = 0
            avail[i,j,k] = 0
            rib[i,j,k] = 1
            n -= 1

In [6]:
import jLM
from jLM.RegionBuilder import RegionBuilder
def fit_ribosomes(nRibosomes,cytoplasm_region=ori_cytoplasm):
    '''
     input: nRibosomes, the number of ribosomes to be placed
     output: fitCytoReg, fitRiboReg
    '''
    avail = RegionBuilder.erode(cytoplasm_region,1).astype(np.int64)
    print(np.unique(avail))
    ribTmp = np.zeros(avail.shape, dtype=np.int64)
    placeRibosomes(nRibosomes, avail, ribTmp)
    print(f"number of ribosomes: {np.sum(ribTmp)}")
    fitRiboReg = np.array(ribTmp, dtype=bool)

    fitCytoReg = cytoplasm_region & ~fitRiboReg
    
    return fitCytoReg, fitRiboReg

save the geometry form RDME-ODE simulation

This is for only let 2102 ribosomes to be available for translation, but we also need to include the rest for obstacles.

In [7]:
effective_ribosomes = 2102
fitCytoRegion, fitRiboRegion = fit_ribosomes(effective_ribosomes,ori_cytoplasm)
rest_ribosomes = 180000 - effective_ribosomes
fitCytoFinalReg, fitObstacleRiboRegion = fit_ribosomes(rest_ribosomes,fitCytoRegion)
import copy
lattice_ribosomes_effective_data = copy.deepcopy(latticeData)
#overwrite the lattice with the new fitCytoRegion and fitRiboRegion
# First add 'ribosome_dummy' to the names list and siteMap, add add to final latticeData.name
latticeData['names'] = list(latticeData['names']) + ['ribosome_dummy']
siteMap['ribosome_dummy'] = len(latticeData['names']) - 1
lattice_ribosomes_effective_data['names'].append('ribosome_dummy')
lattice_ribosomes_effective_data['lattice'][fitCytoFinalReg] = siteMap['cytoplasm']
lattice_ribosomes_effective_data['lattice'][fitRiboRegion] = siteMap['ribosomes']
lattice_ribosomes_effective_data['lattice'][fitObstacleRiboRegion] = siteMap['ribosome_dummy']
# save the new lattice data
with lzma.open("lattice_ribosomes_effective_isolated.pkl.xz", "wb") as f:
    pickle.dump(lattice_ribosomes_effective_data, f)

[0 1]
number of ribosomes: 2102
[0 1]
number of ribosomes: 177898


In [ ]:
break

In [6]:
fitCytoRegion, fitRiboRegion = fit_ribosomes(105000)


import copy
lattice_ribosomes_105000_data = copy.deepcopy(latticeData)
#overwrite the lattice with the new fitCytoRegion and fitRiboRegion
# only those two regions are changed 
lattice_ribosomes_105000_data['lattice'][fitCytoRegion] = siteMap['cytoplasm']
lattice_ribosomes_105000_data['lattice'][fitRiboRegion] = siteMap['ribosomes']

# save the new lattice data
with lzma.open("lattice_ribosomes_105000_isolated.pkl.xz", "wb") as f:
    pickle.dump(lattice_ribosomes_105000_data, f)

[0 1]
number of ribosomes: 105000


In [7]:
fitCytoRegion, fitRiboRegion = fit_ribosomes(180000)


import copy
lattice_ribosomes_180000_data = copy.deepcopy(latticeData)
#overwrite the lattice with the new fitCytoRegion and fitRiboRegion
# only those two regions are changed 
lattice_ribosomes_180000_data['lattice'][fitCytoRegion] = siteMap['cytoplasm']
lattice_ribosomes_180000_data['lattice'][fitRiboRegion] = siteMap['ribosomes']

# save the new lattice data
with lzma.open("lattice_ribosomes_180000_isolated.pkl.xz", "wb") as f:
    pickle.dump(lattice_ribosomes_180000_data, f)


number of ribosomes: 180000


In [8]:
fitCytoRegion, fitRiboRegion = fit_ribosomes(380414)


import copy
lattice_ribosomes_380414_data = copy.deepcopy(latticeData)
#overwrite the lattice with the new fitCytoRegion and fitRiboRegion
# only those two regions are changed 
lattice_ribosomes_380414_data['lattice'][fitCytoRegion] = siteMap['cytoplasm']
lattice_ribosomes_380414_data['lattice'][fitRiboRegion] = siteMap['ribosomes']

# save the new lattice data
with lzma.open("lattice_ribosomes_380414_isolated.pkl.xz", "wb") as f:
    pickle.dump(lattice_ribosomes_380414_data, f)

In [ ]:
# RegionBuilder.showBinaryLattices(new_lattice)